<a href="https://colab.research.google.com/github/asmaakhaledd/PID-NN/blob/optimized-PID-Model/opt_of_PID_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install numpy pandas tensorflow control matplotlib xmltodict scikit-learn

In [18]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Input
import matplotlib.pyplot as plt
from datetime import datetime

Load XML Data

In [35]:
#Automatically loads all XML files (5 training, 5 testing)
train_files = sorted(glob.glob("/content/drive/MyDrive/GP PID/dataset/*-ws-training.xml"))
test_files = sorted(glob.glob("/content/drive/MyDrive/GP PID/dataset/*-ws-testing.xml"))

Check Corrupted XML File

In [36]:
import xml.etree.ElementTree as ET

for file in train_files + test_files:
    try:
        tree = ET.parse(file)
        print(f"✅ {file} is valid")
    except ET.ParseError as e:
        print(f"❌ Error in {file}: {e}")

✅ /content/drive/MyDrive/GP PID/dataset/559-ws-training.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/563-ws-training.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/570-ws-training.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/575-ws-training.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/588-ws-training.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/591-ws-training.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/559-ws-testing.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/563-ws-testing.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/570-ws-testing.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/575-ws-testing.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/588-ws-testing.xml is valid
✅ /content/drive/MyDrive/GP PID/dataset/591-ws-testing.xml is valid


Parse XML Data

In [37]:
#Extracts Time, Glucose, Insulin, and Meal intake
#Sorts data by timestamp to maintain correct sequence order
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    data = []

    # Extract patient weight
    weight = float(root.get('weight', 0))  # Defaults to 0 if missing

    # Store all events for lookup
    meal_events = []
    bolus_events = []
    basal_events = []

    # Extract meal data
    meal_node = root.find('meal')
    if meal_node:
        for event in meal_node.findall('event'):
            meal_events.append({
                'timestamp': datetime.strptime(event.get('ts'), "%d-%m-%Y %H:%M:%S"),
                'carbs': float(event.get('carbs', 0))
            })

    # Extract bolus insulin data
    bolus_node = root.find('bolus')
    if bolus_node:
        for event in bolus_node.findall('event'):
            bolus_events.append({
                'timestamp': datetime.strptime(event.get('ts_begin'), "%d-%m-%Y %H:%M:%S"),
                'dose': float(event.get('dose', 0))
            })

    # Extract basal rate data
    basal_node = root.find('basal')
    if basal_node:
        for event in basal_node.findall('event'):
            basal_events.append({
                'timestamp': datetime.strptime(event.get('ts'), "%d-%m-%Y %H:%M:%S"),
                'rate': float(event.get('value', 0))
            })

    # Ensure all events are sorted by timestamp
    meal_events.sort(key=lambda x: x['timestamp'])
    bolus_events.sort(key=lambda x: x['timestamp'])
    basal_events.sort(key=lambda x: x['timestamp'])

    # Process glucose levels
    for event in root.find('glucose_level'):
        timestamp = datetime.strptime(event.get('ts'), "%d-%m-%Y %H:%M:%S")
        glucose = float(event.get('value'))

        # Find closest meal within 2 hours
        meal_intake = next(
            (m['carbs'] for m in reversed(meal_events) if (timestamp - m['timestamp']).total_seconds() <= 7200),
            0.0
        )

        # Append extracted data
        data.append({
            'timestamp': timestamp,
            'glucose': glucose,
            'meal_carbs': meal_intake,
            'weight': weight
        })

    df = pd.DataFrame(data)

    # Debugging output
    print(f"Parsed {file_path}, {len(df)} records")
    print(df.head(10))  # Show first 10 rows for debugging

    return df.sort_values('timestamp')

# Load all files
train_dfs = [parse_xml(f) for f in train_files]
test_dfs = [parse_xml(f) for f in test_files]

# Concatenate all data
train_df = pd.concat(train_dfs, ignore_index=True)
test_df = pd.concat(test_dfs, ignore_index=True)

# Final debug prints
print("\nFinal Train DataFrame:")
print(train_df.head(10))
print("\nFinal Test DataFrame:")
print(test_df.head(10))

Parsed /content/drive/MyDrive/GP PID/dataset/559-ws-training.xml, 10796 records
            timestamp  glucose  meal_carbs  weight
0 2021-12-07 01:17:00    101.0        40.0    99.0
1 2021-12-07 01:22:00     98.0        40.0    99.0
2 2021-12-07 01:27:00    104.0        40.0    99.0
3 2021-12-07 01:32:00    112.0        40.0    99.0
4 2021-12-07 01:37:00    120.0        40.0    99.0
5 2021-12-07 01:42:00    127.0        40.0    99.0
6 2021-12-07 01:47:00    135.0        40.0    99.0
7 2021-12-07 01:52:00    142.0        40.0    99.0
8 2021-12-07 01:57:00    140.0        40.0    99.0
9 2021-12-07 02:02:00    145.0        40.0    99.0
Parsed /content/drive/MyDrive/GP PID/dataset/563-ws-training.xml, 12124 records
            timestamp  glucose  meal_carbs  weight
0 2021-09-13 12:33:00    219.0        45.0    99.0
1 2021-09-13 12:38:00    229.0        45.0    99.0
2 2021-09-13 12:43:00    224.0        45.0    99.0
3 2021-09-13 12:48:00    221.0        45.0    99.0
4 2021-09-13 12:53:00   

Convert Time into Numerical Features

In [38]:
# Convert Time into Cyclic Features (Sin/Cos encoding)
def preprocess_time_features(df):
    df['hour'] = df['timestamp'].dt.hour
    df['minute'] = df['timestamp'].dt.minute
    df['time_sin'] = np.sin(2 * np.pi * df['hour'] / 24)  # Cyclic encoding
    df['time_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    return df.drop(['timestamp', 'hour', 'minute'], axis=1)

train_df = preprocess_time_features(train_df)
test_df = preprocess_time_features(test_df)

Convert all data to float32

In [39]:
train_df = train_df.astype(np.float32)
test_df = test_df.astype(np.float32)

Prepare sequences for the PID tuning model

In [40]:
def prepare_pid_training_data(df, sequence_length=30):
    X_pid, y_pid = [], []
    for i in range(len(df) - sequence_length):
        glucose_error = df['glucose'].iloc[i] - 110  # Setpoint = 110 mg/dL (target glucose)
        glucose_change = df['glucose'].iloc[i] - df['glucose'].iloc[i-1] if i > 0 else 0

        # Kp, Ki, Kd are based on glucose error and change (scaled)
        Kp = 0.05 * np.log(1 + abs(glucose_error))
        Ki = 0.005 * np.log(1 + abs(glucose_error))
        Kd = 0.002 * np.log(1 + abs(glucose_change))

        # Add features (glucose_error, glucose_change, meal_intake, weight, time_sin, time_cos)
        X_pid.append([glucose_error, glucose_change, df['meal_carbs'].iloc[i], df['weight'].iloc[i],
                      df['time_sin'].iloc[i], df['time_cos'].iloc[i]])  # Adjust for cyclic time encoding
        y_pid.append([Kp, Ki, Kd])  # Target PID gains

    return np.array(X_pid), np.array(y_pid)

X_pid_train, y_pid_train = prepare_pid_training_data(train_df)
X_pid_test, y_pid_test = prepare_pid_training_data(test_df)


Define LSTM Model for PID Parameter Tuning

In [44]:
pid_input = Input(shape=(X_pid_train.shape[1], 1))
pid_lstm = LSTM(64, activation='tanh', return_sequences=True)(pid_input)
pid_lstm = LSTM(32, activation='tanh')(pid_lstm)
pid_output = Dense(3, activation='linear')(pid_lstm)  # Predict Kp, Ki, Kd

pid_model = Model(inputs=pid_input, outputs=pid_output)
pid_model.compile(optimizer='adam', loss='mse')

Reshape X_pid train  and X_pid test for lstm

In [45]:
X_pid_train_reshaped = X_pid_train.reshape((X_pid_train.shape[0], X_pid_train.shape[1], 1))  # (samples, timesteps, features)
X_pid_test_reshaped = X_pid_test.reshape((X_pid_test.shape[0], X_pid_test.shape[1], 1))  # (samples, timesteps, features)

Train and save the model

In [46]:
pid_model.fit(X_pid_train_reshaped, y_pid_train, epochs=50, batch_size=32, validation_data=(X_pid_test_reshaped, y_pid_test))
pid_model.save("/content/drive/MyDrive/GP PID/opt_pid_tuning_model.h5")

Epoch 1/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 3.4874e-04 - val_loss: 7.6059e-06
Epoch 2/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 16s 7ms/step - loss: 1.1099e-05 - val_loss: 4.1510e-06
Epoch 3/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 7.3581e-06 - val_loss: 1.7946e-06
Epoch 4/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 4.3691e-06 - val_loss: 3.3106e-06
Epoch 5/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 3.8820e-06 - val_loss: 2.9847e-06
Epoch 6/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 2.4814e-06 - val_loss: 1.0641e-06
Epoch 7/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 2.1548e-06 - val_loss: 1.8567e-06
Epoch 8/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 1.2591e-06 - val_loss: 2.4082e-06
Epoch 9/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - loss: 1.1953e-06 - val_loss: 2.9680e-06
Epoch 10/50
2164/2164 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 1.2466e-06 - val_loss: 7.3859e-07
Epoch 11/50
2164/2164 ━━━━━━━

Save and Load the model

In [47]:
import tensorflow as tf

# Define the loss function explicitly
pid_model = tf.keras.models.load_model(
    "/content/drive/MyDrive/GP PID/opt_pid_tuning_model.h5",
    custom_objects={'mse': tf.keras.losses.MeanSquaredError()}
)

# Print model summary to confirm it's loaded
pid_model.summary()
import tensorflow as tf

# Load the model with safe mode
pid_model = tf.keras.models.load_model("/content/drive/MyDrive/GP PID/opt_pid_tuning_model.h5", compile=False)

# Compile again with correct loss
pid_model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, 6, 1)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 6, 64)               │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 29,413 (114.90 KB)

 Trainable params: 29,411 (114.89 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)



```
# This is formatted as code
```

Load pid model

In [48]:
pid_model = tf.keras.models.load_model("/content/drive/MyDrive/GP PID/opt_pid_tuning_model.h5", compile=False)

In [49]:
#test
pid_gains = pid_model.predict(X_pid_test)

499/499 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Test the model

In [67]:
num_samples = 50  # Number of test cases to evaluate

for i in range(num_samples):
    print(f"\nSample {i+1}:")

    # Ensure correct input shape for LSTM (batch size 1)
    sample_input = np.expand_dims(X_pid_test[i], axis=0)

    # Predict PID gains (Kp, Ki, Kd)
    pid_gains = pid_model.predict(sample_input)[0]

    # Get the actual glucose value from the test dataset
    glucose_prediction = X_pid_test[i][0]  # Assuming glucose is the first feature in the input sequence

    # Compute glucose error (current glucose - target glucose)
    glucose_error = glucose_prediction - 110  # Target glucose level = 110 mg/dL

    # Compute the change in glucose (derivative of glucose)
    if i > 0:
        glucose_change = glucose_prediction - X_pid_test[i-1][0]  # Compare current and previous glucose
    else:
        glucose_change = 0  # For the first sample, assume no change

    weight = test_df['weight'].iloc[i]
    # Insulin Sensitivity Factor (ISF) - Adjust per patient
    ISF = 5000 / weight

    # Compute Correction Dose Using ISF
    glucose_correction_insulin = glucose_error / ISF  # Correct glucose error

    # Compute Insulin Using PID Formula
    # Apply a safe correction factor and limit the insulin dosage based on target glucose levels
    # Reduce PID gains and scaling factor
    insulin_dosage = (pid_gains[0] * glucose_error +  # Proportional (Kp)
                  pid_gains[1] * glucose_error +  # Integral (Ki)
                  pid_gains[2] * glucose_change) * 0.1  # Derivative (Kd) - Reduced scaling factor

    # Apply insulin correction for the glucose error
    insulin_dosage += glucose_correction_insulin

    # Ensure the insulin dosage is within a reasonable range (e.g., max 10U per dose)
    # Dynamic max insulin limit
    max_insulin = min(10 + (glucose_prediction - 110) / 10, 20)  # Adjust the max insulin based on glucose
    insulin_dosage = max(0, min(insulin_dosage + glucose_correction_insulin, max_insulin))


    # Print results
    print(f"Predicted Glucose: {glucose_prediction:.2f} mg/dL")
    print(f"Recommended Insulin: {insulin_dosage:.2f} U (Safe Range)")
    print(f"Weight: {weight}")

    print("-" * 50)



Sample 1:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Predicted Glucose: 69.00 mg/dL
Recommended Insulin: 0.00 U (Safe Range)
Weight: 99.0
--------------------------------------------------

Sample 2:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Predicted Glucose: 73.00 mg/dL
Recommended Insulin: 0.00 U (Safe Range)
Weight: 99.0
--------------------------------------------------

Sample 3:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Predicted Glucose: 77.00 mg/dL
Recommended Insulin: 0.00 U (Safe Range)
Weight: 99.0
--------------------------------------------------

Sample 4:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted Glucose: 81.00 mg/dL
Recommended Insulin: 0.00 U (Safe Range)
Weight: 99.0
--------------------------------------------------

Sample 5:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Predicted Glucose: 85.00 mg/dL
Recommended Insulin: 0.00 U (Safe Range)
Weight: 99.0
--------------------------------------------------

Sample 6:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Predicted Glucose: 89.00 m